In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 26 21:08:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             48W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [6]:
!pip install datasets

In [5]:
!pip install setfit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [3]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from datasets import DatasetDict, Dataset
from setfit import SetFitModel,Trainer,TrainingArguments
import os

In [ ]:
train_df = pd.read_csv("train.csv")  # Contains 'text' and 'label' (string)
test_df = pd.read_csv("test.csv")

In [ ]:
# Sample 20 utterances per label
sampled_train_df = (
    train_df.groupby("label", group_keys=False)
    .apply(lambda x: x.sample(n=7, random_state=42))
    .reset_index(drop=True)
)

# Sample 20 utterances per label
sampled_test_df = (
    test_df.groupby("label", group_keys=False)
    .apply(lambda x: x.sample(n=5, random_state=42))
    .reset_index(drop=True)
)

<ipython-input-20-93f2b48e3619>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=7, random_state=42))
<ipython-input-20-93f2b48e3619>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=5, random_state=42))


In [ ]:
print(sampled_train_df.shape)
print(sampled_test_df.shape)

(539, 2)
(385, 2)


In [ ]:
train_dataset = Dataset.from_pandas(sampled_train_df)
test_dataset = Dataset.from_pandas(sampled_test_df)

In [ ]:
unique_labels = sorted(train_dataset.unique("label"))

In [ ]:
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L12-v2",labels=unique_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    batch_size=32,
    num_epochs=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    metric="accuracy",
    column_mapping={"text": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/539 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

***** Running training *****
  Num unique pairs = 286748
  Batch size = 32
  Num epochs = 2


Epoch,Training Loss,Validation Loss
1,0.000200,0.075283
2,0.000100,0.077830


In [ ]:
val_preds = trainer.model.predict(dataset["validation"]["text"])
val_labels = dataset["validation"]["label"]

val_acc = accuracy_score(val_labels, val_preds)
val_f1 = f1_score(val_labels, val_preds, average="macro")

print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")

NameError: name 'dataset' is not defined

In [ ]:
unseen_dataset = Dataset.from_pandas(test_df)
test_preds = trainer.model.predict(unseen_dataset["text"])
test_labels = unseen_dataset["label"]

test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average="macro")

print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Test Accuracy: 0.7851
Test F1 Score: 0.7832


In [ ]:
type(test_preds)

numpy.ndarray

In [ ]:
df_dict = {'text': unseen_dataset['text'], 'label': unseen_dataset['label'], 'pred_label': test_preds.tolist()}

predicted_unseen_df = pd.DataFrame(df_dict)

In [ ]:
predicted_unseen_df.head(5)

,text,label,pred_label
0,How do I locate my card?,card_arrival,card_linking
1,"I still have not received my new card, I order...",card_arrival,card_arrival
2,I ordered a card but it has not arrived. Help ...,card_arrival,card_arrival
3,Is there a way to know when my card will arrive?,card_arrival,card_delivery_estimate
4,My card has not arrived yet.,card_arrival,card_arrival


In [ ]:
filepath = '/content/drive/My Drive/output_folder/predicted_unseen_df_banking.xlsx' # Replace 'your_folder' and 'your_file'
predicted_unseen_df.to_excel(filepath, index=False)

print(f"DataFrame saved to: {filepath}")

DataFrame saved to: /content/drive/My Drive/output_folder/predicted_unseen_df_banking.xlsx


In [ ]:
trainer.model.save_pretrained("./fine-tuned-banking-model")

In [ ]:
model.save_pretrained("/content/drive/MyDrive/banking_setfit_model")

## Visualization: Pre vs Post

In [15]:
! pip install plotly

In [16]:
! pip install umap-learn

In [17]:
from sentence_transformers import SentenceTransformer
import umap
import plotly.express as px
from sklearn.preprocessing import LabelEncoder

In [8]:
filepath = '/content/drive/My Drive/output_folder/predicted_unseen_df_banking.xlsx' # Replace 'your_folder' and 'your_file'
predicted_unseen_df=pd.read_excel(filepath)

In [14]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(predicted_unseen_df['text'], show_progress_bar=True)

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

In [19]:
# Reduce Dimensions with UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
embedding_2d = reducer.fit_transform(embeddings)

# Label encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(predicted_unseen_df['label'])
label_names = label_encoder.classes_

# Create DataFrame for Plotly
df = pd.DataFrame({
    'x': embedding_2d[:, 0],
    'y': embedding_2d[:, 1],
    'label': predicted_unseen_df['label'],
    'utterance': predicted_unseen_df['text']
})

# Create interactive plot
fig = px.scatter(
    df, x='x', y='y',
    color='label',
    hover_data=['utterance'],
    title='Utterance Embeddings (UMAP)',
    width=1000, height=800
)

fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [20]:
model = SentenceTransformer('/content/drive/MyDrive/banking_setfit_model')
embeddings = model.encode(predicted_unseen_df['text'], show_progress_bar=True)

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

In [21]:
# Reduce Dimensions with UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
embedding_2d = reducer.fit_transform(embeddings)

# Label encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(predicted_unseen_df['pred_label'])
label_names = label_encoder.classes_

# Create DataFrame for Plotly
df = pd.DataFrame({
    'x': embedding_2d[:, 0],
    'y': embedding_2d[:, 1],
    'label': predicted_unseen_df['pred_label'],
    'utterance': predicted_unseen_df['text']
})

# Create interactive plot
fig = px.scatter(
    df, x='x', y='y',
    color='label',
    hover_data=['utterance'],
    title='Utterance Embeddings (UMAP)',
    width=1000, height=800
)

fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

